In [46]:
# Importação de bibliotecas
import pandas as pd
import numpy as np

In [47]:
# Variáveis utilizadas
dataset_name = 'risco_erosao'

raiz = '/home/eduardo/repositorios/IS_Agro/databases'

In [48]:
# Tabelas necessárias para o desenvolvimento
df_risco_pastagem = pd.read_csv(f'{raiz}/ouro_{dataset_name}_pastagem.csv').convert_dtypes()
df_risco_lav_perm = pd.read_csv(f'{raiz}/ouro_{dataset_name}_lavouras_perenes.csv').convert_dtypes()
df_risco_lav_temp = pd.read_csv(f'{raiz}/ouro_{dataset_name}_lavouras_temporarias.csv').convert_dtypes()

In [49]:
df_risco_erosao = (
    pd.concat([
        pd.read_csv(f'{raiz}/ouro_{dataset_name}_pastagem.csv').convert_dtypes(),
        pd.read_csv(f'{raiz}/ouro_{dataset_name}_lavouras_perenes.csv').convert_dtypes(),
        pd.read_csv(f'{raiz}/ouro_{dataset_name}_lavouras_temporarias.csv').convert_dtypes()
    ])
    .assign(
        level = lambda df: np.where(df.sub_label < 6, 'Tolerável Erosão',
                                    np.where((df.sub_label >= 6) & (df.sub_label < 11), 'Baixa Erosão',
                                             np.where((df.sub_label >= 11) & (df.sub_label < 22), 'Moderada Erosão',
                                                      np.where((df.sub_label >= 22) & (df.sub_label < 33), 'Alta Erosão',
                                                               np.where(df.sub_label >= 33, 'Severa Erosão',
                                                                        pd.NA)))))
    )
    .groupby(['country', 'state', 'date', 'level'])
    .agg({'value':'sum'})
    .convert_dtypes()
    .reset_index()
    .rename(columns={'level': 'label'})
)

In [50]:
df_risco_erosao.to_csv(f'{raiz}/ouro_{dataset_name}.csv', index=False)